In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# PyMuPDF: PDF에서 텍스트를 빠르고 정확하게 추출하기 위한 라이브러리입니다.
# "camelot-py[cv]": PDF 속 표(Table)를 인식하고 구조화된 데이터로 추출하기 위한 라이브러리입니다. '[cv]'는 표 인식을 위한 추가 기능(OpenCV)을 함께 설치하라는 뜻입니다.
# pandas: Camelot이 추출한 표 데이터를 다루기 쉬운 형태(DataFrame)로 만들어주고, 마크다운으로 변환할 때 사용합니다.
!pip install PyMuPDF "camelot-py[cv]" pandas

#데이터 처리 자동화
라이브러리 임포트 및 파일 경로 설정

In [ ]:
import fitz  # PyMuPDF 라이브러리를 사용할 때는 'fitz'라는 이름으로 불러옵니다.
import camelot
import pandas as pd
import os      # os는 'Operating System'의 약자로, 폴더를 만들거나 파일 목록을 읽어오는 등 파일 시스템 작업을 돕습니다.

# --- 사용자 설정 영역 ---
# 본인의 구글 드라이브에 맞게 경로를 수정해주세요!
BASE_DIR = '/content/drive/MyDrive/RAG_Audit_Project'

# 1. 원본 PDF 파일들이 저장된 폴더 경로
SOURCE_PDF_DIR = os.path.join(BASE_DIR, '01_source_pdfs')

# 2. 정제된 텍스트 파일들을 저장할 폴더 경로
CLEANED_TEXT_DIR = os.path.join(BASE_DIR, '02_cleaned_text')

# 출력 폴더가 존재하지 않으면 자동으로 생성합니다.
os.makedirs(CLEANED_TEXT_DIR, exist_ok=True)

print(f"PDF 원본 폴더: {SOURCE_PDF_DIR}")
print(f"텍스트 저장 폴더: {CLEANED_TEXT_DIR}")

/usr/local/lib/python3.11/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


PDF 원본 폴더: /content/drive/MyDrive/RAG_Audit_Project/01_source_pdfs
텍스트 저장 폴더: /content/drive/MyDrive/RAG_Audit_Project/02_cleaned_text


 PDF 텍스트 및 표 추출 함수 정의

In [ ]:
def extract_text_from_pdf(pdf_path):
    print(f"PDF 파일 처리 시작: {os.path.basename(pdf_path)}")

    # 1. PDF 열기
    doc = fitz.open(pdf_path)

    # 2. 전체 텍스트를 저장할 변수 초기화
    full_text = ""

    # 3. 모든 페이지를 하나씩 순회
    for page_num, page in enumerate(doc):
        # page_num은 0부터 시작하는 페이지 번호입니다.
        print(f"  - {page_num + 1}/{len(doc)} 페이지 처리 중...")

        # 3-1. 해당 페이지의 일반 텍스트 추출
        full_text += page.get_text()
        full_text += "\n\n" # 페이지 구분을 위해 줄바꿈 추가

        # 3-2. Camelot을 이용해 해당 페이지의 표 추출
        try:
            # flavor='lattice'는 표의 선을 기준으로 추출하는 옵션으로, 일반적인 표에 효과적입니다.
            tables = camelot.read_pdf(pdf_path, pages=str(page_num + 1), flavor='lattice')

            # 표가 성공적으로 추출되었다면
            if tables:
                for i, table in enumerate(tables):
                    # 추출된 표(DataFrame)를 LLM이 이해하기 좋은 마크다운 형식으로 변환
                    markdown_table = table.df.to_markdown(index=False)

                    # 일반 텍스트와 구분하기 위해 표의 시작과 끝을 명시
                    full_text += f"\n--- 표 시작 (페이지: {page_num + 1}, 표: {i+1}) ---\n"
                    full_text += markdown_table
                    full_text += f"\n--- 표 끝 ---\n\n"
        except Exception as e:
            # Camelot이 표를 찾지 못하거나 오류가 발생해도 전체 작업이 중단되지 않도록 예외 처리
            print(f"    (경고) {page_num+1} 페이지에서 표를 추출하지 못했습니다: {e}")

    # 4. 열었던 PDF 파일 닫기
    doc.close()

    print(f"PDF 파일 처리 완료: {os.path.basename(pdf_path)}")
    return full_text

모든 PDF 파일 일괄 처리 및 저장

In [ ]:
# 1. 원본 PDF 폴더에 있는 모든 파일 목록을 가져옵니다.
try:
    pdf_files = [f for f in os.listdir(SOURCE_PDF_DIR) if f.lower().endswith('.pdf')]
except FileNotFoundError:
    print(f"오류: '{SOURCE_PDF_DIR}' 폴더를 찾을 수 없습니다. 경로를 확인해주세요.")
    pdf_files = [] # 오류가 나면 빈 리스트로 초기화

# 2. PDF 파일 목록을 순회하며 하나씩 처리
if not pdf_files:
    print("처리할 PDF 파일이 없습니다. '01_source_pdfs' 폴더에 파일을 넣어주세요.")
else:
    print(f"\n총 {len(pdf_files)}개의 PDF 파일을 처리합니다.")

    for pdf_filename in pdf_files:
        # 2-1. 입출력 파일의 전체 경로를 생성
        input_pdf_path = os.path.join(SOURCE_PDF_DIR, pdf_filename)

        # 저장될 텍스트 파일의 이름은 원본 PDF 이름에서 확장자만 .txt로 변경
        output_txt_filename = os.path.splitext(pdf_filename)[0] + '.txt'
        output_txt_path = os.path.join(CLEANED_TEXT_DIR, output_txt_filename)

        # 2-2. 이미 정제된 텍스트 파일이 존재하는지 확인하고, 존재하면 건너뜁니다.
        if os.path.exists(output_txt_path):
            print(f"⏩ 건너뛰기: '{output_txt_filename}' 파일이 이미 존재합니다.")
            continue # 다음 파일로 넘어갑니다.

        # 2-3. 위에서 정의한 함수를 호출하여 텍스트 추출 실행
        extracted_content = extract_text_from_pdf(input_pdf_path)

        # 2-4. 추출된 내용을 텍스트 파일로 저장
        # 'w'는 쓰기 모드, encoding='utf-8'은 한글이 깨지지 않도록 보장하는 필수 옵션입니다.
        with open(output_txt_path, 'w', encoding='utf-8') as f:
            f.write(extracted_content)

        print(f"✅ 성공: '{output_txt_path}' 에 저장 완료!\n")

print("--- 모든 작업이 완료되었습니다! ---")


총 3개의 PDF 파일을 처리합니다.
⏩ 건너뛰기: 'source_audit_godroute.pdf.txt' 파일이 이미 존재합니다.
⏩ 건너뛰기: 'source_audit_overview.pdf.txt' 파일이 이미 존재합니다.
⏩ 건너뛰기: 'source_audit_handbook.pdf.txt' 파일이 이미 존재합니다.
--- 모든 작업이 완료되었습니다! ---


# 이미지 페이지만 선별하여 OCR 추가 실행하기

 1단계: Tesseract OCR 설치

In [ ]:
# Tesseract OCR 엔진과 한글 언어팩을 Colab 환경에 설치합니다.
!sudo apt install tesseract-ocr tesseract-ocr-kor
# Tesseract를 파이썬에서 쉽게 사용하기 위한 라이브러리를 설치합니다.
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-kor is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


특정 페이지만 OCR 처리하는 코드

In [ ]:
# --- 라이브러리 임포트 및 경로 재설정 ---
import fitz
import pytesseract
from PIL import Image
import io
import os

BASE_DIR = '/content/drive/MyDrive/RAG_Audit_Project'
SOURCE_PDF_DIR = os.path.join(BASE_DIR, '01_source_pdfs')
CLEANED_TEXT_DIR = os.path.join(BASE_DIR, '02_cleaned_text')
# --- 경로 설정 끝 ---

In [ ]:
def ocr_specific_pages(pdf_path, page_numbers):
    # PDF 파일과 특정 페이지 번호 목록을 받아 해당 페이지만 OCR 처리하는 함수.
    doc = fitz.open(pdf_path)
    ocr_texts = ""
    print(f"  - 총 {len(page_numbers)}개의 이미지 페이지에 대해 OCR을 시작합니다...")
    for page_num in page_numbers:
        page = doc.load_page(page_num)
        pix = page.get_pixmap(dpi=300) # 페이지를 고해상도 이미지로 변환
        img_bytes = pix.tobytes("png")
        image = Image.open(io.BytesIO(img_bytes))

        # Tesseract를 이용해 한글 텍스트 추출
        text = pytesseract.image_to_string(image, lang='kor')

         # 추출된 텍스트를 결과에 추가 (어느 페이지인지 명시)
        ocr_texts += f"\n\n--- [OCR 추가] 페이지: {page_num + 1} ---\n"
        ocr_texts += text
    doc.close()
    return ocr_texts

In [ ]:
# --- 메인 실행 로직 (건너뛰기 기능 수정) ---
try:
    pdf_files = [f for f in os.listdir(SOURCE_PDF_DIR) if f.lower().endswith('.pdf')]
except FileNotFoundError:
    print(f"오류: '{SOURCE_PDF_DIR}' 폴더를 찾을 수 없습니다. 경로를 확인해주세요.")
    pdf_files = []

if not pdf_files:
    print("처리할 PDF 파일이 없습니다.")
else:
    for pdf_filename in pdf_files:
        print(f"\n--- '{pdf_filename}' 파일의 이미지 페이지를 확인합니다 ---")
        input_pdf_path = os.path.join(SOURCE_PDF_DIR, pdf_filename)
        output_txt_filename = os.path.splitext(pdf_filename)[0] + '.txt'
        output_txt_path = os.path.join(CLEANED_TEXT_DIR, output_txt_filename)

        # 이미 OCR 처리가 완료된 파일인지 확인
        if os.path.exists(output_txt_path):
            try:
                with open(output_txt_path, 'r', encoding='utf-8') as f:
                    # 파일의 '전체' 내용을 읽어서 OCR 표식이 있는지 확인합니다.
                    content = f.read()
                    if "--- [OCR 추가] 페이지:" in content:
                        print(f"⏩ 건너뛰기: '{output_txt_filename}' 파일은 이미 OCR 처리가 완료되었습니다.")
                        continue # continue를 만나면 아래 코드를 실행하지 않고 바로 다음 파일로 넘어갑니다.
            except Exception as e:
                print(f"  - 경고: '{output_txt_filename}' 파일 확인 중 오류 발생: {e}. 건너뛰지 않고 처리합니다.")



--- 'source_audit_godroute.pdf.pdf' 파일의 이미지 페이지를 확인합니다 ---
⏩ 건너뛰기: 'source_audit_godroute.pdf.txt' 파일은 이미 OCR 처리가 완료되었습니다.

--- 'source_audit_overview.pdf.pdf' 파일의 이미지 페이지를 확인합니다 ---
⏩ 건너뛰기: 'source_audit_overview.pdf.txt' 파일은 이미 OCR 처리가 완료되었습니다.

--- 'source_audit_handbook.pdf.pdf' 파일의 이미지 페이지를 확인합니다 ---
⏩ 건너뛰기: 'source_audit_handbook.pdf.txt' 파일은 이미 OCR 처리가 완료되었습니다.


In [ ]:
        # PDF를 열어 텍스트가 거의 없는 페이지(이미지 추정)를 찾습니다.
        image_based_pages = []
        try:
            doc = fitz.open(input_pdf_path)
            for page_num, page in enumerate(doc):
                text = page.get_text()
                if len(text.strip()) < 100:
                    image_based_pages.append(page_num)
            doc.close()

            if image_based_pages:
                print(f"  - 이미지 추정 페이지: {[p + 1 for p in image_based_pages]}")
                ocr_result = ocr_specific_pages(input_pdf_path, image_based_pages)

                with open(output_txt_path, 'a', encoding='utf-8') as f:
                    f.write(ocr_result)
                print(f"✅ OCR 결과를 '{output_txt_filename}' 파일에 성공적으로 추가했습니다.")
            else:
                print("  - 추가로 처리할 이미지 페이지가 없습니다.")
        except Exception as e:
            print(f"  - 파일 처리 중 오류 발생: {e}")

print("\n--- 모든 파일의 이미지 페이지 처리가 완료되었습니다! ---")

  - 이미지 추정 페이지: [1, 2, 3, 4, 5, 6, 7, 8, 16, 59, 74, 75, 216, 217, 285, 287, 288, 289, 290, 291]
  - 총 20개의 이미지 페이지에 대해 OCR을 시작합니다...


KeyboardInterrupt: 

#핵심 RAG 파이프라인
RAG 파이프라인 라이브러리 설치

In [ ]:
# langchain-core, langchain: LangChain의 핵심 기능을 담고 있는 라이브러리입니다.
# langchain_openai: OpenAI의 언어 모델(LLM)과 임베딩 모델을 LangChain에서 사용하게 해주는 연결고리입니다.
# langchain_community: 다양한 외부 도구(Loader, Vectorstore 등)와의 연동을 지원하는 커뮤니티 라이브러리입니다.
# tiktoken: OpenAI 모델이 텍스트를 처리하는 방식(토큰화)에 맞춰 글자 수를 계산하고 자르기 위해 필요합니다.
# faiss-cpu: 텍스트를 변환한 숫자 벡터(Vector)를 효율적으로 저장하고 빠르게 검색하기 위한 데이터베이스입니다. 'cpu' 버전입니다.
!pip install langchain-core langchain langchain_openai langchain_community tiktoken faiss-cpu

OpenAI API 키 설정

In [ ]:
# 1. Colab 왼쪽 메뉴에서 '🔑' (보안 비밀) 아이콘을 클릭합니다.
# 2. '새 보안 비밀 추가'를 누릅니다.
# 3. 이름(name)은 'OPENAI_API_KEY'로, 값(value)에는 발급받은 본인의 API 키('sk-...'로 시작)를 붙여넣습니다.
# 4. '노트북 액세스' 토글을 켜줍니다.

import os
from google.colab import userdata

# userdata.get() 함수를 통해 저장된 API 키를 안전하게 불러옵니다.
# 이렇게 하면 코드에 키가 직접 보이지 않아 Github 등에 공유해도 안전합니다.
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

print("OpenAI API 키 설정이 완료되었습니다.")

OpenAI API 키 설정이 완료되었습니다.


텍스트 로드, 분할 및 벡터 DB 구축

In [ ]:
# 1. PDF 및 이미지 처리를 위한 필수 시스템 라이브러리를 설치합니다.
# - poppler-utils: PDF 파일의 텍스트나 이미지를 추출하는 데 사용됩니다.
# - tesseract-ocr, tesseract-ocr-kor: 이미지 속 글자를 인식(OCR)하는 데 필요한 엔진과 한글 팩입니다.
# - libreoffice: .doc, .pptx 등 MS Office 계열 파일을 처리하는 데 도움을 줍니다.
!apt-get install -y libreoffice poppler-utils tesseract-ocr tesseract-ocr-kor


# 버전 충돌을 해결하기 위해 pypdf 버전을 camelot-py에 맞게 지정하여 설치합니다.
!pip install PyMuPDF "camelot-py[cv]" pandas "pypdf>=3.17,<4.0" unstructured

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-kor is already the newest version (1:4.00~git30-7274cfa-1.1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
libreoffice is already the newest version (1:7.3.7-0ubuntu0.22.04.10).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from langchain_community.document_loaders import DirectoryLoader # 폴더 내의 모든 문서를 한번에 불러옵니다.
from langchain.text_splitter import RecursiveCharacterTextSplitter # 텍스트를 재귀적으로 자르는 도구입니다.
from langchain_openai import OpenAIEmbeddings # 텍스트를 벡터로 변환하는 모델입니다.
from langchain_community.vectorstores import FAISS # 벡터를 저장하고 검색하는 DB입니다.

# --- 경로 설정 ---
# 텍스트 파일들이 저장된 폴더 경로
CLEANED_TEXT_DIR = '/content/drive/MyDrive/RAG_Audit_Project/02_cleaned_text'
# 생성된 벡터 DB를 저장할 폴더 경로
VECTOR_STORE_DIR = '/content/drive/MyDrive/RAG_Audit_Project/03_vector_store'
os.makedirs(VECTOR_STORE_DIR, exist_ok=True) # 폴더가 없으면 생성



1. 데이터 로드 (Load)

In [ ]:
# DirectoryLoader를 사용해 CLEANED_TEXT_DIR 폴더 안의 모든 .txt 파일을 불러옵니다.
print("1. 텍스트 파일들을 불러오는 중...")
loader = DirectoryLoader(CLEANED_TEXT_DIR, glob="**/*.txt")
documents = loader.load()
print(f"  - 총 {len(documents)}개의 문서를 불러왔습니다.")



1. 텍스트 파일들을 불러오는 중...
  - 총 3개의 문서를 불러왔습니다.


2. 텍스트 분할 (Split)

In [ ]:
# 문서를 너무 잘게 나누면 맥락이 사라지고, 너무 크게 나누면 관련 없는 정보가 섞입니다.
# chunk_size: 나눌 텍스트의 최대 크기 (토큰 기준)
# chunk_overlap: 잘린 부분들이 서로 겹치는 크기. 문맥이 끊어지는 것을 방지합니다.
print("2. 문서를 의미있는 조각(chunk)으로 분할하는 중...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
split_documents = text_splitter.split_documents(documents)
print(f"  - 총 {len(split_documents)}개의 조각으로 분할되었습니다.")

2. 문서를 의미있는 조각(chunk)으로 분할하는 중...
  - 총 1184개의 조각으로 분할되었습니다.


3. 임베딩 및 벡터 DB 구축 (Embed & Store), 벡터 DB파일 저장

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import os
import time # 시간 지연을 위한 time 모듈 임포트

# --- 경로 설정 ---
CLEANED_TEXT_DIR = '/content/drive/MyDrive/RAG_Audit_Project/02_cleaned_text'
VECTOR_STORE_DIR = '/content/drive/MyDrive/RAG_Audit_Project/03_vector_store'
os.makedirs(VECTOR_STORE_DIR, exist_ok=True)

# 1. 데이터 로드 (Load)
print("1. 텍스트 파일들을 불러오는 중...")
loader = DirectoryLoader(CLEANED_TEXT_DIR, glob="**/*.txt")
documents = loader.load()
print(f"  - 총 {len(documents)}개의 문서를 불러왔습니다.")

# 2. 텍스트 분할 (Split)
print("2. 문서를 의미있는 조각(chunk)으로 분할하는 중...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
split_documents = text_splitter.split_documents(documents)
print(f"  - 총 {len(split_documents)}개의 조각으로 분할되었습니다.")

# 3. 임베딩 모델 로드
print("3. 임베딩 모델을 로드하는 중...")
embeddings = OpenAIEmbeddings() # OpenAIEmbeddings 객체 생성

# 4. 벡터 DB 구축 (Embed & Store) - 배치 처리 및 add_embeddings 사용
print("4. 텍스트 조각을 벡터로 변환하고 DB를 구축하는 중... (시간이 다소 걸릴 수 있습니다)")

# API 요청 한도를 고려하여 문서를 작은 배치로 나눕니다.
# 30만 토큰 한도 및 4만 TPM 한도를 고려하여 배치 크기를 설정합니다.
batch_size = 100 # 배치 크기 (이전 시도와 동일하게 100으로 유지)
delay_seconds = 5 # 배치 처리 후 대기 시간 (초) - 필요에 따라 조절하세요.

# 첫 번째 배치를 사용하여 FAISS 인덱스 생성
print(f"  - 첫 번째 {min(batch_size, len(split_documents))}개 조각 처리 중...")
first_batch_documents = split_documents[:batch_size]
vectorstore = FAISS.from_documents(first_batch_documents, embeddings)
print(f"  - 첫 번째 배치 벡터화 및 DB 구축 완료.")

# 나머지 배치 처리 및 기존 인덱스에 추가
for i in range(batch_size, len(split_documents), batch_size):
    print(f"  - {i+1} ~ {min(i+batch_size, len(split_documents))} 조각 처리 중...")
    batch_documents = split_documents[i : i + batch_size]

    # 배치 문서의 텍스트 내용 추출
    batch_texts = [doc.page_content for doc in batch_documents]

    # 배치 텍스트를 임베딩 벡터로 변환
    # RateLimitError를 피하기 위해 이 호출 전후에 시간 지연을 추가합니다.
    try:
        batch_embeddings = embeddings.embed_documents(batch_texts)
    except Exception as e:
        print(f"  - 임베딩 중 오류 발생: {e}. 잠시 후 재시도합니다.")
        time.sleep(10) # 오류 발생 시 10초 대기 후 재시도
        batch_embeddings = embeddings.embed_documents(batch_texts) # 재시도

    # FAISS 인덱스에 문서 내용과 벡터 추가
    # add_embeddings 메서드는 텍스트 리스트와 해당 임베딩 벡터 리스트를 인자로 받습니다.
    vectorstore.add_embeddings(list(zip(batch_texts, batch_embeddings))) # zip으로 텍스트와 벡터를 묶어서 전달
    print(f"  - 배치 처리 및 DB 추가 완료.")

    # 분당 토큰 한도를 초과하지 않도록 배치 처리 후 대기
    print(f"  - {delay_seconds}초 대기 중...")
    time.sleep(delay_seconds)

# 5. 벡터 DB 파일로 저장
vectorstore.save_local(VECTOR_STORE_DIR)

print(f"✅ 성공: 벡터 데이터베이스를 '{VECTOR_STORE_DIR}' 폴더에 저장 완료!")

1. 텍스트 파일들을 불러오는 중...
  - 총 3개의 문서를 불러왔습니다.
2. 문서를 의미있는 조각(chunk)으로 분할하는 중...
  - 총 1184개의 조각으로 분할되었습니다.
3. 임베딩 모델을 로드하는 중...
4. 텍스트 조각을 벡터로 변환하고 DB를 구축하는 중... (시간이 다소 걸릴 수 있습니다)
  - 첫 번째 100개 조각 처리 중...
  - 첫 번째 배치 벡터화 및 DB 구축 완료.
  - 101 ~ 200 조각 처리 중...
  - 배치 처리 및 DB 추가 완료.
  - 5초 대기 중...
  - 201 ~ 300 조각 처리 중...
  - 배치 처리 및 DB 추가 완료.
  - 5초 대기 중...
  - 301 ~ 400 조각 처리 중...
  - 배치 처리 및 DB 추가 완료.
  - 5초 대기 중...
  - 401 ~ 500 조각 처리 중...
  - 배치 처리 및 DB 추가 완료.
  - 5초 대기 중...
  - 501 ~ 600 조각 처리 중...
  - 배치 처리 및 DB 추가 완료.
  - 5초 대기 중...
  - 601 ~ 700 조각 처리 중...
  - 배치 처리 및 DB 추가 완료.
  - 5초 대기 중...
  - 701 ~ 800 조각 처리 중...
  - 배치 처리 및 DB 추가 완료.
  - 5초 대기 중...
  - 801 ~ 900 조각 처리 중...
  - 배치 처리 및 DB 추가 완료.
  - 5초 대기 중...
  - 901 ~ 1000 조각 처리 중...
  - 배치 처리 및 DB 추가 완료.
  - 5초 대기 중...
  - 1001 ~ 1100 조각 처리 중...
  - 배치 처리 및 DB 추가 완료.
  - 5초 대기 중...
  - 1101 ~ 1184 조각 처리 중...
  - 배치 처리 및 DB 추가 완료.
  - 5초 대기 중...
✅ 성공: 벡터 데이터베이스를 '/content/drive/MyDrive/RAG_Audit_Project/03_vector_store' 폴더에 저